# Tutorial 2: Hello MPI World

**Content creators**: Jan Ebert, Stefan Kesselheim

**Content reviewers / testers**: Alexandre Strube

After learning how to connect to the supercomputer and start jobs in the last tutorial, let us now create our own more complex jobs taking advantage of the distributed system by using the [_Message Passing Interface (MPI)_ standard](https://www.mpi-forum.org/). You will implement some parallel programs which require different compute nodes to split their work and communicate with one another.

## Exercise 1: Parallel Hello World

For getting to know the basic concepts of MPI, you will now implement a simple, distributed 'hello world'-program using the [_mpi4py_](https://bitbucket.org/mpi4py/mpi4py) library ([link to documentation](https://mpi4py.readthedocs.io/en/stable/); it's sparse, so use the Python `help(function)` command if you need more information).

In MPI, we always have access to the communicator for our processes through the `MPI.COMM_WORLD` variable. The type of `MPI.COMM_WORLD` is `MPI.Comm`, the communicator class. The communicator can tell us, for example, which process we reside in (`MPI.Comm.Get_rank()`) or the total number of processes in our "world" (`MPI.Comm.Get_size()`). We would call these methods on our `MPI.COMM_WORLD`, like `MPI.COMM_WORLD.Get_rank()`.\
These are already all the tools you need for a typical MPI 'hello world'!

An MPI program works by spawning multiple copies of the same program in different processes which are distributed over the nodes in our cluster. 
Because MPI uses a special environment to provide these features, our program needs extra setup. Slurm handles this for us with the `srun` command. In order to execute an MPI-enabled batch job via Slurm, we would write the following `sbatch` script:
```sh
#!/usr/bin/env bash
#SBATCH --nodes=4
#SBATCH --ntasks-per-node=1
# [...]

source /p/project/training2306/software_environment/activate.sh

srun python hello_world.py
```
This would run a distributed `hello_world.py` on 4 different nodes, giving us a world size of 4. The `--ntasks-per-node` flag specifies how many independent executions should be started on each node. You'll get a feel for what this means in the following tasks.

### Tasks
0. Take a look at and understand the `exercise1.sbatch` script.
1. Extend the code in `exercise1.py` to print a line like "I am a unique process with rank {rank}." for each process.
1. Let a single "root" process summarize how many processes exist in total so we do not miss one.
1. Vary the `--ntasks-per-node` flag in your `exercise1.sbatch`.
1. Vary the `--nodes` flag in your `exercise1.sbatch`.

Also, if you already know MPI from other languages, you will notice the familiar `MPI.Init()` and `MPI.Finalize()` statements are not required. mpi4py automatically initializes MPI and registers an exit handler for `MPI.Finalize` on the first import.

## Exercise 2: Collective Communication

Communication is important! Luckily, just like us, MPI has this functionality built in. Several functions based around passing messages between different nodes are provided by the interface. One of these is `MPI.Comm.Gather`, an operation which _gathers_ individual results into a larger one (for example, integers into an array of integers).
Most of the communication functions expect as parameters the data to be sent away and a buffer to store received data in. Both of these parameters usually need a type association based on MPI primitives (like `MPI.INT` or `MPI.DOUBLE`) so that MPI knows how much data to send and expect. Luckily, MPI has strong support for Python's [NumPy](https://numpy.org/) array library and can determine the datatype by itself if passed a NumPy array.

With this knowledge, let us take a look at the signature of `MPI.Comm.Gather`, where we will see one other concept related to communication:
```python
MPI.Comm.Gather(self, sendbuf, recvbuf, root=0)
```
The `root` parameter indicates the rank of the process that our data in `sendbuf` (send buffer) is sent towards. This is also the only process which actually uses the `recvbuf` (receive buffer) we pass in, meaning for other processes it can just as well be `None`. Not all communication functions use a root process, but the ones we will treat in this tutorial all do.

So, with `MPI.Comm.Gather`, we obtain our result by _gathering_ individual process results into one larger result (for example integers into an array of integers). MPI has another trick up its sleeve to _reduce_ results into a result of the same size (for example integers into an integer): `MPI.Comm.Reduce`.\
`MPI.Comm.Reduce` works very similar to `MPI.Comm.Gather`, providing a root process with a combined result of the individually communicated data. We pass an additional parameter, the _reduction_ operation, for which MPI provides several primitives. For example:
- `MPI.SUM` and `MPI.PROD` to calculate a sum and product
- `MPI.MIN` and `MPI.MAX` to find the minimum and maximum value
- `MPI.LAND` and `MPI.LOR` to compute the equivalents of `all` and `any` using `and` and `or` (`&&` and `||` in C)

We will now try out the `MPI.Comm.Gather` and `MPI.Comm.Reduce` operations by calculating π stochastically.

We provide the following algorithm (there's a more detailed description at the bottom of the notebook if you are interested after the tutorial):

1. Sample a large amount ($n$) of points in the square for which $x, y \in [0, 1]$.
1. Find out how many points are inside the unit circle ($x^2 + y^2 \leq 1$); that amount is $m$.
1. Calculate the ratio $r = m \div n$.
1. Multiply the ratio by 4: $r \times 4 \approx \pi$.

Check out the following example for $n = 10$:
![π algorithm example](pi_example.png)
[(link if image not showing)](pi_example.png)

We can easily see that $m = 8$ points are in the circle. Thus, we have $r = m \div n = 8 \div 10 = 0.8$, which results in $\pi \approx r \times 4 = 3.2$ – not bad!

This problem can easily be parallelized: each process samples its own points, does its own containment checks, and finally submits the number of points inside the circle ($m$) to the root process. Because visualization is also a big part of deep learning, we are going to plot all the sampled points in distinct colors indicating the process they were sampled in. This means we need to gather all the points we sample in _one_ big array in the _one_ process which plots the results.

### Tasks
1. Create the file `exercise2.sbatch`, using `exercise1.sbatch` as a foundation.
1. Inside `exercise2.py`, use `MPI.Comm.Reduce` to fuse the calculations distributed to multiple processes. Please use the receive buffer `total_num_in_circle` that is already set up for you (we will calculate the ratio later to avoid floating point error accumulation).
1. With `MPI.Comm.Gather`, create a large array of all sampled points so we can plot them. Please use the receive buffer `all_samples` that is already set up.

## Exercise 3: Sharding a Dataset

The previous exercise was pretty simple in regards to distributing the workload. Each process generates the required samples. With typical machine learning datasets, we need to distribute existing data over multiple processes so that each process handles non-overlapping data. This splitting up of data is called _sharding_. Each process gets only a _shard_ of the full data.

For our exercise, we will use the [Tiny-ImageNet-200](http://cs231n.stanford.edu/tiny-imagenet-200.zip) dataset (we already downloaded this for you). It contains 120&#8239;000 images depicting 200 different categories. The 120&#8239;000 images are split up into 100&#8239;000 images for training, 10&#8239;000 images for validation during training and 10&#8239;000 images for testing. You will use a reduction operation to distributively calculate a histogram over this dataset. The histogram is a 255×3 NumPy matrix containing the number of intensity values for each pixel in the image for each color channel (red, green and blue). 

### Tasks
1. Find `# Task 1` in `exercise3.py` and implement sharding for the image paths in the dataset. All processes should receive the same number of objects (except for one process if the number of images is not divisible by the number of processes) and we do not want to leave out any data.
1. Find `# Task 2` and use one of the two MPI communication functions we introduced to sum up the histograms from each process.
1. We work on the zipped file instead of extracting it beforehand. Can you think of a reason other than saving disk space why that is?
1. In the provided code, each process obtains the paths to the images it is going to handle by itself by filtering a list of all files in the dataset.\
   Would it make more sense to have one process filter the paths and distribute them to each process? Why or why not? Think of cases where either method has its advantages.
   
#### Hints
* To select elements in a Python list, you can use the array slicing operator `[begin:end:skip]`. It will create a sub-list containing the elements from index `begin` until (excluding) index `end`, using an increment of `skip`. Negative values indicate an index counted from the end (-1 is the last value in the list). Each entry in the operator can be left out, then they default to `begin=0`, `end=len(list)`, `skip=1`. This is most easily seen in a few examples:

In [2]:
my_list = list(range(9))
print("my_list", my_list)
print("my_list[2:5]", my_list[2:5])
print("my_list[:-1]", my_list[:-1])
print("my_list[::2]", my_list[::2])
print("my_list[1::2]", my_list[1::2])

my_list [0, 1, 2, 3, 4, 5, 6, 7, 8]
my_list[2:5] [2, 3, 4]
my_list[:-1] [0, 1, 2, 3, 4, 5, 6, 7]
my_list[::2] [0, 2, 4, 6, 8]
my_list[1::2] [1, 3, 5, 7]


## Appendix: Explanation of the π Algorithm

Imagine a square with side lengths of 2 that has a circle inside. The circle's diameter is as large as the square's edge length, so the circle snuggly fits into the square, just touching its four edges. Were we to randomly take points from anywhere inside the square, we would sometimes get points that lie inside the circle as well. Other points may _not_ lie inside the circle. Sampling many random points in the square, we can obtain a good estimate for the ratio of the area of the circle (which we know is $r^2 \times \pi = 1^2 \times \pi = \pi$) in relation to the area of the square (which we know is $s^2 = 2^2 = 4$) without knowing π. The ratio is thus $A_{circle} \div A_{square} = \pi \div 4$.

Assuming we didn't know about π, how could we calculate the area of the circle? Since we are able to easily calculate the area of the square, we can multiply the area ratio from above by the area of the square, obtaining the area of the circle. If we use random sampling (or a grid of samples) to obtain an approximate ratio as in the exercise, we can approximate the circle area and – by extension – π.

We can even simplify this a bit by dividing the square into a 2 by 2 window, where each part of the window contains a quarter circle. Sampling from one of the window parts does not change the ratio (as both the square and the circle are quartered), but to obtain the full circle area, we will still have to multiply by 4 (due to only approximating the area of a quarter circle). Finally, we can define the algorithm from the exercise that only samples in the smaller area.